In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

import graphviz
from graphviz import Graph

import env
import wrangle_zillow
import os

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

SyntaxError: invalid syntax (wrangle_zillow.py, line 538)

In [ ]:
# change display settings to show all columns
pd.set_option("display.max_columns", None)

In [ ]:
# use a function to pull in zillow data
df = wrangle_zillow.wrangle_zillow()
df.shape

In [ ]:
# use a function to split data for exploring and modeling
train, validate, test = wrangle_zillow.split_data(df)
train.head(3)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe().T

In [ ]:
# check bathroom value counts
train.bathrooms.value_counts().plot.bar()

In [ ]:
# check bedroom value counts
train.bedrooms.value_counts().plot.bar()

In [ ]:
# check binned area value counts
train.area.value_counts(bins=10).plot.bar()

In [ ]:
# check binned logerror value counts
train.logerror.value_counts(bins=10).plot.bar()

In [ ]:
# check binned logerror value counts
train.counties.value_counts().plot.bar()

In [ ]:
# for col in train.columns:
#     plt.figure(figsize=(4,2))
#     plt.hist(train[col])
#     plt.title(col)
#     plt.show()

### Takeaways:

#### Many of the features seem to be skewed to the right

#### Bathrooms, bedrooms, area, and age have the highest correlation with logerror

#### There are almost twice as many properties in LA County than Orange & Ventura county

## Is there difference in mean logerror for each of the counties?

In [ ]:
train.head(3)

In [ ]:
# check mean logerror for each county
train.groupby('counties').logerror.mean()

In [ ]:
# code is not producing the same result as groupby likely due to rounding
train[train.counties == 'los_angeles'].logerror.mean()

In [ ]:
# code is not producing the same result as groupby likely due to rounding
train[train.counties == 'orange'].logerror.mean()

In [ ]:
# code is not producing the same result as groupby likely due to rounding
train[train.counties == 'ventura'].logerror.mean()

**My project partner explored this in greater depth with statistical testing and determined there was a significant difference in logerror by county**

In [ ]:
# Use .describe with object columns.
obj_cols = train.columns[[train[col].dtype == 'O' for col in train.columns]]
for col in obj_cols:
    print(train[col].value_counts())
    print(train[col].value_counts(normalize=True, dropna=False))
    print('----------------------')

## What does absolute logerror look like from county to county?

In [ ]:
# check the range for logerror
train.logerror.describe()

In [ ]:
# add a column that bins each value of logerror into max, min, or med absolute error
train['log_e'] = pd.cut(train.logerror, bins=[-5,-1,-.03,.03,1,5], ordered=False, labels=['max','med','min','med','max'])
train.head()

In [ ]:
# plot the data to see which areas have the most logerror
sns.relplot(data=train, x='latitude', y='longitude', hue='log_e', hue_order=['max', 'med','min'], height=10, palette='rocket')

### As seen towards the center of the graph, Los Angeles County does have a higher proportion of med - max logerror which could mean the model has a harder time predicting home values from this location or could just be due to the larger number of properties sold in this area

In [ ]:
# use crosstab to visualize the number of each category per county
pd.crosstab(train.counties, train.log_e)

In [ ]:
# plot the data to see if any county has a higher percentage of max logerrors
x, y, hue = 'counties', 'proportion', 'log_e'
hue_order = ['max', 'med', 'min']

(train[hue]
 .groupby(train[x])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue))

### We can barely see on the graph but it looks like the model produces a slightly higher percentage of max errors for Orange County but a higher percentage of medium error for Los Angeles County

In [ ]:
# drop log_e to prep for clustering
train = train.drop(columns='log_e')
train.head()

In [ ]:
# use a function to create X and y datasets for train, validate, and test
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle_zillow.prep_zillow_for_model(train, validate, test)
X_train.head(2)

In [ ]:
y_train.isnull().sum()

In [ ]:
# select the features to use
X = X_train[['age', 'taxvalue']]
X2 = X_validate[['age', 'taxvalue']]
X3 = X_test[['age', 'taxvalue']]

In [ ]:
# visualize distribution
X.hist()

In [ ]:
# visualize data to see if there are any obvious clusters
sns.relplot(x = 'age', y ='taxvalue', data = X, height=8)

**There are no obvious clusters so I will use the elbow method to see if I can find a good value for k**

In [ ]:
# use elbow method to see what might be a good value for k
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k).fit(X).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

### Based on this visualization, I will start by using a k of 5 since the slope starts tapering off after that

In [ ]:
# use KMeans to create cluster

# define the thing
kmeans = KMeans(n_clusters=5, random_state = 369)

# fit the thing
kmeans.fit(X)

# Use the thing to predict
kmeans.predict(X)

In [ ]:
# create a new column with the predicted cluster in the original X_dataframes
X_train['agetax_cluster'] = kmeans.predict(X)
X_validate['agetax_cluster'] = kmeans.predict(X2)
X_test['agetax_cluster'] = kmeans.predict(X3)
X_train.head(2)

In [ ]:
X_validate.head(2)

In [ ]:
# create dataframe of cluster centers
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)
centroids

In [ ]:
# visualize clustering results
sns.relplot(x = 'age', y ='taxvalue', data = X_train, hue = 'agetax_cluster')

centroids.plot.scatter(x='age', y='taxvalue', c='black', marker='x', s=1000, ax=plt.gca(), label='centroid', figsize=(10, 8))

In [ ]:
# concatenate X and y train dataframes to plot data
Xy_train = pd.concat([X_train, y_train], axis=1)

# visualize distribution of logerror by cluster
plt.figure(figsize=(12,10))
sns.boxplot(
    x='agetax_cluster',
    y='logerror',
    data=Xy_train)

In [ ]:
# use KMeans to create 3 clusters to see if that may be more meaningful
# define the thing
kmeans = KMeans(n_clusters=3, random_state = 369)

# fit the thing
kmeans.fit(X)

# Use the thing to predict
kmeans.predict(X)

# create a new column with the predicted cluster in the original X_dataframes
X_train['agetax_cluster'] = kmeans.predict(X)
X_validate['agetax_cluster'] = kmeans.predict(X2)
X_test['agetax_cluster'] = kmeans.predict(X3)

# create dataframe of cluster centers
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)

# visualize clustering results
sns.relplot(x = 'age', y ='taxvalue', data = X_train, hue = 'agetax_cluster')

centroids.plot.scatter(x='age', y='taxvalue', c='black', marker='x', s=1000, ax=plt.gca(), label='centroid', figsize=(10, 8))

In [ ]:
# visualize distribution of logerror by cluster
plt.figure(figsize=(12,10))
sns.boxplot(
    x='agetax_cluster',
    y='logerror',
    data=Xy_train)

### Although the visual does not make it appear that there is much of a difference in logerror for each cluster, I am going to apply a statistical test to confirm this.

In [ ]:
y_train.isnull().sum()

In [ ]:
# concatenate X_train and y_train so I can check variance of logerror by cluster
X_y = pd.concat([X_train, y_train], axis=1)
X_y.head(2)

In [ ]:
X_y.logerror.value_counts(dropna=False)

**The null hypothesis for the levene test is that there is equal variance in logerror for the clusters**

**The alternate hypothesis for the levene test is that there is unequal variance in logerror for the clusters**

In [ ]:
# set alpha
alpha = 0.05

# use levene test to check variance of each cluster
stat, pvalue = stats.levene(
                            X_y[X_y.agetax_cluster == 0].logerror,
                            X_y[X_y.agetax_cluster == 1].logerror,
                            X_y[X_y.agetax_cluster == 2].logerror)

print(f'{stat}, {pvalue}')
if pvalue > alpha:
    print('We fail to reject the null hypothesis')
elif pvalue < alpha:
    print('We reject the null hypothesis')

**Because of unequal variance, I will use the Kruskal-Wallis test to compare the median logerror for each cluster**

**The null hypothesis is that there is no significant difference in the median logerror for each of the clusters**

**The alternate hypothesis is that there is a significant difference in the median logerror for each of the clusters**

In [ ]:
# set alpha
alpha = 0.05

# use kruskal-wallis test to compare medians
stat, pvalue = stats.kruskal(
    X_y[X_y.agetax_cluster == 0].logerror,
    X_y[X_y.agetax_cluster == 1].logerror,
    X_y[X_y.agetax_cluster == 2].logerror)

print(f'{stat}, {pvalue}')
if pvalue > alpha:
    print('We fail to reject the null hypothesis')
elif pvalue < alpha:
    print('We reject the null hypothesis')

### It appears there is a statistically significant difference in logerror for at least two of the clusters

**I am going to use selectkbest and recursive feature elimination to see what features might be most relevant**

In [ ]:
# parameters: f_regression stats test, give me 3 features
f_selector = SelectKBest(f_regression, k=3)

# find the top 3 X's correlated with y
f_selector.fit(X_train, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train.iloc[:,feature_mask].columns.tolist()
f_feature

In [ ]:
# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, n_features_to_select=3)

# fit the data using RFE
rfe.fit(X_train,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()
rfe_feature

In [ ]:
# select the features to use
X = X_train[['bathrooms', 'bedrooms', 'area']]
X2 = X_validate[['bathrooms', 'bedrooms', 'area']]
X3 = X_test[['bathrooms', 'bedrooms', 'area']]

In [ ]:
# visualize distribution for X
X.hist()

In [ ]:
# visualize data to see if there are any obvious clusters
sns.relplot(x = 'bathrooms', y ='area', hue='bedrooms', data = train, height=6)

In [ ]:
# use elbow method to see what might be a good value for k
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k).fit(X).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

In [ ]:
# use KMeans to create 4 clusters

# define the thing
kmeans = KMeans(n_clusters=4, random_state = 369)

# fit the thing
kmeans.fit(X)

# Use the thing to predict
kmeans.predict(X)

# create a new column with the predicted cluster in the original X_train
X_train['bedbath_area_cluster'] = kmeans.predict(X)
X_validate['bedbath_area_cluster'] = kmeans.predict(X2)
X_test['bedbath_area_cluster'] = kmeans.predict(X3)

# create dataframe of cluster centers
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)

# visualize clustering results
sns.relplot(x = 'bathrooms', y ='area', data = X_train, hue = 'bedrooms', col='bedbath_area_cluster')

In [ ]:
# visualize distribution of logerror by cluster
plt.figure(figsize=(12,10))
sns.boxplot(
    x='bedbath_area_cluster',
    y=y_train,
    data=X_train)

In [ ]:
# concatenate X_train and y_train so I can check variance of logerror by cluster
X_y = pd.concat([X_train, y_train], axis=1)
X_y.head(2)

**The null hypothesis for the levene test is that there is equal variance in logerror for the clusters**

**The alternate hypothesis for the levene test is that there is unequal variance in logerror for the clusters**

In [ ]:
# set alpha
alpha = 0.05

# use levene test to check variance of each cluster
stat, pvalue = stats.levene(
                            X_y[X_y.bedbath_area_cluster == 0].logerror,
                            X_y[X_y.bedbath_area_cluster == 1].logerror,
                            X_y[X_y.bedbath_area_cluster == 2].logerror,
                            X_y[X_y.bedbath_area_cluster == 2].logerror)

print(f'{stat}, {pvalue}')
if pvalue > alpha:
    print('We fail to reject the null hypothesis')
elif pvalue < alpha:
    print('We reject the null hypothesis')

**Because of unequal variance, I will use the Kruskal-Wallis test to compare the mean logerror for each cluster**

**The null hypothesis is that there is no significant difference in the median logerror for each of the clusters**

**The alternate hypothesis is that there is a significant difference in the median logerror for each of the clusters**

In [ ]:
# set alpha
alpha = 0.05

# use kruskal-wallis test to compare medians
stats.kruskal(
    X_y[X_y.bedbath_area_cluster == 0].logerror,
    X_y[X_y.bedbath_area_cluster == 1].logerror,
    X_y[X_y.bedbath_area_cluster == 2].logerror)

print(f'{stat}, {pvalue}')
if pvalue > alpha:
    print('We fail to reject the null hypothesis')
elif pvalue < alpha:
    print('We reject the null hypothesis')

In [ ]:
# use groupby to check logerror stats for each of the clusters
X_y.groupby('bedbath_area_cluster').logerror.agg(['min', 'median','mean', 'max'])

In [ ]:
X_train.head(3)

In [ ]:
# give clusters names
X_train.agetax_cluster = X_train.agetax_cluster.map({0: "older_lowtaxvalue",
                                                     1: "newer_lowtaxvalue",
                                                     2: "all_ages_hightaxvalue"})

X_validate.agetax_cluster = X_validate.agetax_cluster.map({0: "older_lowtaxvalue",
                                                           1: "newer_lowtaxvalue",
                                                           2: "all_ages_hightaxvalue"})

X_test.agetax_cluster = X_test.agetax_cluster.map({0: "older_lowtaxvalue",
                                                     1: "newer_lowtaxvalue",
                                                     2: "all_ages_hightaxvalue"})

X_train.bedbath_area_cluster = X_train.bedbath_area_cluster.map({0: "large_3plusbed",
                                                                 1: "small_2bed",
                                                                 2: "tiny_1bed",
                                                                 3: "medium_3bed"})

X_validate.bedbath_area_cluster = X_validate.bedbath_area_cluster.map({0: "large_3plusbed",
                                                                       1: "small_2bed",
                                                                       2: "tiny_1bed",
                                                                       3: "medium_3bed"})

X_test.bedbath_area_cluster = X_test.bedbath_area_cluster.map({0: "large_3plusbed",
                                                                 1: "small_2bed",
                                                                 2: "tiny_1bed",
                                                                 3: "medium_3bed"})

In [ ]:
X_train.agetax_cluster.value_counts()

In [ ]:
# encode cluster columns
X_train_model = pd.get_dummies(X_train[['agetax_cluster','bedbath_area_cluster']])
X_validate_model = pd.get_dummies(X_validate[['agetax_cluster','bedbath_area_cluster']])
X_test_model = pd.get_dummies(X_test[['agetax_cluster','bedbath_area_cluster']])
X_train_model.head()

In [ ]:
y_train.head(3)

### First I am going to calculate RMSE for predicting median as the baseline. This will give us something to evaluate our other models against

In [ ]:
# turn y for each dataset from a series to a dataframe
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

# create a baseline
y_train['baseline'] = y_train.logerror.median()
y_validate['baseline'] = y_train.logerror.median()
y_test['baseline'] = y_train.logerror.median()

# 4. RMSE of logerror median
rmse_train_baseline = mean_squared_error(y_train.logerror, y_train.baseline)**(1/2)
rmse_validate_baseline = mean_squared_error(y_validate.logerror, y_validate.baseline)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train_baseline, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_baseline, 5))

### In order to beat baseline, any of the models would need to have a RMSE of less than 0.02197 on the train dataset and 0.02068 on the validate dataset.

## Ordinary Least Squares Model

In [ ]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data and specify y column 
lm.fit(X_train_model, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lm'] = lm.predict(X_train_model)
y_validate['logerror_pred_lm'] = lm.predict(X_validate_model)

# evaluate train & validate: rmse
rmse_train_OLS = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm)**(1/2)
rmse_validate_OLS = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", round(rmse_train_OLS, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_OLS, 5))

## LassoLars Model

In [ ]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data and specify y column 
lars.fit(X_train_model, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lars'] = lars.predict(X_train_model)
y_validate['logerror_pred_lars'] = lars.predict(X_validate_model)

# evaluate train & validate: rmse
rmse_train_lars = mean_squared_error(y_train.logerror, y_train.logerror_pred_lars)**(1/2)
rmse_validate_lars = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lars)**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", round(rmse_train_lars, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_lars, 5))

**I tried different values of alpha from .01 to 1.5 and received the same results with all of them.**

## Tweedie Regressor (GLM) Model

In [ ]:
y_train.logerror.hist()

In [ ]:
# create the model object
glm = TweedieRegressor(power=0, alpha=0.01)

# fit the model to our training data and specify y column 
glm.fit(X_train_model, y_train.logerror)

# predict train & validate
y_train['logerror_pred_glm'] = glm.predict(X_train_model)
y_validate['logerror_pred_glm'] = glm.predict(X_validate_model)

# evaluate train & validate: rmse
rmse_train_glm = mean_squared_error(y_train.logerror, y_train.logerror_pred_glm)**(1/2)
rmse_validate_glm = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_glm)**(1/2)

print("RMSE for GLM using Tweedie, power=0 & alpha=1\nTraining/In-Sample: ", round(rmse_train_glm, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_glm, 5))

**I tried different values for power and alpha from .01 to 1 and achieved the best result with a smaller value of alpha**

## Polynomial Regression

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_model
X_train_degree2 = pf.fit_transform(X_train_model)

# transform X_validate_model & X_test_model
X_validate_degree2 = pf.transform(X_validate_model)
X_test_degree2 = pf.transform(X_test_model)

In [ ]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data and specify y column 
lm2.fit(X_train_degree2, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lm2'] = lm2.predict(X_train_degree2)
y_validate['logerror_pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_train_pr = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm2)**(1/2)
rmse_validate_pr = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm2)**(1/2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", round(rmse_train_pr, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_pr, 5))

In [ ]:
X_train_model.head(2)

### Let's try a subset of the features to see if we get a better result

In [ ]:
# select only bed, bath, area cluster features
X_train_bba = X_train_model.iloc[:,3:]
X_validate_bba = X_validate_model.iloc[:,3:]
X_train_bba.head()

In [ ]:
# create the model object
lm3 = LinearRegression(normalize=True)

# fit the model to our training data and specify y column 
lm3.fit(X_train_bba, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lm3'] = lm3.predict(X_train_bba)
y_validate['logerror_pred_lm3'] = lm3.predict(X_validate_bba)

# evaluate train & validate: rmse
rmse_train_sub1 = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm3)**(1/2)
rmse_validate_sub1 = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm3)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", round(rmse_train_sub1, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_sub1, 5))

In [ ]:
# select only age, tax cluster features
X_train_agetax = X_train_model.iloc[:,:3]
X_validate_agetax = X_validate_model.iloc[:,:3]
X_train_agetax.head()

In [ ]:
# create the model object
lm4 = LinearRegression(normalize=True)

# fit the model to our training data and specify y column 
lm4.fit(X_train_agetax, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lm4'] = lm4.predict(X_train_agetax)
y_validate['logerror_pred_lm4'] = lm4.predict(X_validate_agetax)

# evaluate train & validate: rmse
rmse_train_sub2 = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm4)**(1/2)
rmse_validate_sub2 = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm4)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", round(rmse_train_sub2, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_sub2, 5))

### Let's try using latitude and longitude as features for modeling using OLS & Polynomial model

In [ ]:
# select features to use for modeling
X_train_latlong = X_train[['latitude','longitude']]
X_validate_latlong = X_validate[['latitude','longitude']]
X_train_latlong.head(3)

In [ ]:
# create the model object
lm5 = LinearRegression(normalize=True)

# fit the model to our training data and specify y column 
lm5.fit(X_train_latlong, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lm5'] = lm5.predict(X_train_latlong)
y_validate['logerror_pred_lm5'] = lm5.predict(X_validate_latlong)

# evaluate train & validate: rmse
rmse_train_sub3 = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm5)**(1/2)
rmse_validate_sub3 = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm5)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", round(rmse_train_sub3, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_sub3, 5))

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_model
X_train_degree2 = pf.fit_transform(X_train_latlong)

# transform X_validate_model & X_test_model
X_validate_degree2 = pf.transform(X_validate_latlong)
# X_test_degree2 = pf.transform(X_test_model)

In [ ]:
# create the model object
lm6 = LinearRegression(normalize=True)

# fit the model to our training data and specify y column 
lm6.fit(X_train_degree2, y_train.logerror)

# predict train & validate
y_train['logerror_pred_lm6'] = lm6.predict(X_train_degree2)
y_validate['logerror_pred_lm6'] = lm6.predict(X_validate_degree2)

# evaluate train & validate: rmse
rmse_train_sub3_pr = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm6)**(1/2)
rmse_validate_sub3_pr = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm6)**(1/2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", round(rmse_train_sub3_pr, 5), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate_sub3_pr, 5))

## The best model is Tweedie Regressor followed by OLS using the encoded bedbath_area and age_tax features.